# Main.py


Before running this code, you need to create a env file and set the DATA_DIR environment variable.
You can do so by running the following command:
```
echo "DATA_DIR=/path/to/your/data" > .env
```

In [1]:
import os
from dotenv import load_dotenv
import glob
from pathlib import Path
import subprocess
# Load environment variables from .env file
load_dotenv(".env")

# Get the data directory path
os.getenv('DATA_DIR')

'/mnt/e/30-1040147467'

In [2]:
# Get the data directory path
data_dir = os.getenv('DATA_DIR')

if not data_dir:
    raise ValueError("DATA_DIR environment variable is not set in .env file")

# Expand the home directory if present
data_dir = os.path.expanduser(data_dir)

print(f"Data directory path: {data_dir}")

# Verify if the directory exists
if not os.path.exists(data_dir):
    raise FileNotFoundError(f"Directory not found: {data_dir}")
else:
    print(f"Directory exists: {data_dir}")

Data directory path: /mnt/e/30-1040147467
Directory exists: /mnt/e/30-1040147467


The next step is to create the output directories.

In [4]:
# Create output directories
output_dirs = {
    'fastqc': os.path.join(data_dir, '01_fastqc'),
    'trimmed': os.path.join(data_dir, '02_trimmed'),
    'star': os.path.join(data_dir, '03_star'),
    'counts': os.path.join(data_dir, '04_counts')
}

# Create directories if they don't exist
for dir_name, dir_path in output_dirs.items():
    os.makedirs(dir_path, exist_ok=True)
    print(f"Created directory: {dir_path}")

# Get list of R1 fastq files
fastq_dir = os.path.join(data_dir, '00_fastq')
r1_files = sorted(glob.glob(os.path.join(fastq_dir, '*_R1_001.fastq.gz')))

# Print sample information
print("\nSample information:")
for r1_file in r1_files:
    sample_name = os.path.basename(r1_file).replace('_R1_001.fastq.gz', '')
    r2_file = r1_file.replace('_R1_001.fastq.gz', '_R2_001.fastq.gz')
    print(f"Sample: {sample_name}")
    print(f"  R1: {os.path.basename(r1_file)}")
    print(f"  R2: {os.path.basename(r2_file)}")

Created directory: /mnt/e/30-1040147467/01_fastqc
Created directory: /mnt/e/30-1040147467/02_trimmed
Created directory: /mnt/e/30-1040147467/03_star
Created directory: /mnt/e/30-1040147467/04_counts

Sample information:
Sample: 1
  R1: 1_R1_001.fastq.gz
  R2: 1_R2_001.fastq.gz
Sample: 2
  R1: 2_R1_001.fastq.gz
  R2: 2_R2_001.fastq.gz
Sample: 3
  R1: 3_R1_001.fastq.gz
  R2: 3_R2_001.fastq.gz
Sample: 4
  R1: 4_R1_001.fastq.gz
  R2: 4_R2_001.fastq.gz
Sample: 5
  R1: 5_R1_001.fastq.gz
  R2: 5_R2_001.fastq.gz


In [ ]:
# Define cutadapt parameters
cutadapt_params = [
    '--minimum-length', '20',  # Minimum read length after trimming
    '--quality-cutoff', '20',  # Quality score cutoff (based on your report's quality metrics)
    '--cores', '4',           # Number of CPU cores to use
    '--pair-filter=any',      # Remove both reads if either is filtered
    '-a', 'AGATCGGAAGAGCACACGTCTGAACTCCAGTCA',  # TruSeq Universal Adapter
    '-A', 'AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT',  # TruSeq Indexed Adapter
    '--trim-n',              # Remove Ns from the ends
]

# Process each sample
for r1_file in r1_files:
    sample_name = os.path.basename(r1_file).replace('_R1_001.fastq.gz', '')
    r2_file = r1_file.replace('_R1_001.fastq.gz', '_R2_001.fastq.gz')
    
    # Define output files
    out_r1 = os.path.join(output_dirs['trimmed'], f'{sample_name}_R1_trimmed.fastq.gz')
    out_r2 = os.path.join(output_dirs['trimmed'], f'{sample_name}_R2_trimmed.fastq.gz')
    log_file = os.path.join(output_dirs['trimmed'], f'{sample_name}_cutadapt.log')
    
    print(f"Processing sample {sample_name}...")
    
    # Run cutadapt
    cmd = ['cutadapt'] + cutadapt_params + [
        '-o', out_r1,
        '-p', out_r2,
        r1_file,
        r2_file
    ]
    
    # Run the command and capture output
    with open(log_file, 'w') as f:
        result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        f.write(result.stdout)
        f.write(result.stderr)
    
    if result.returncode == 0:
        print(f"Successfully processed {sample_name}")
    else:
        print(f"Error processing {sample_name}")
        print(result.stderr)

Processing sample 1...
Error processing 1
Run "cutadapt --help" to see command-line options.
See https://cutadapt.readthedocs.io/ for full documentation.

cutadapt: error: unrecognized arguments: --interleaved-output

Processing sample 2...
Error processing 2
Run "cutadapt --help" to see command-line options.
See https://cutadapt.readthedocs.io/ for full documentation.

cutadapt: error: unrecognized arguments: --interleaved-output

Processing sample 3...
Error processing 3
Run "cutadapt --help" to see command-line options.
See https://cutadapt.readthedocs.io/ for full documentation.

cutadapt: error: unrecognized arguments: --interleaved-output

Processing sample 4...
Error processing 4
Run "cutadapt --help" to see command-line options.
See https://cutadapt.readthedocs.io/ for full documentation.

cutadapt: error: unrecognized arguments: --interleaved-output

Processing sample 5...
Error processing 5
Run "cutadapt --help" to see command-line options.
See https://cutadapt.readthedocs.io

This code will:
- Use cutadapt to trim the reads
- Remove Illumina TruSeq adapters
- Trim low-quality bases (quality score < 20)
- Remove reads shorter than 20bp
- Process both R1 and R2 reads together
- Save trimmed reads and log files in the 02_trimmed directory

In [5]:
# Create genome directory structure
genome_dir = os.path.join(data_dir, 'genome')
os.makedirs(genome_dir, exist_ok=True)

# Define URLs for genome and annotation files
genome_url = "https://ftp.ensembl.org/pub/release-110/fasta/mus_musculus/dna/Mus_musculus.GRCm39.dna.primary_assembly.fa.gz"
gtf_url = "https://ftp.ensembl.org/pub/release-110/gtf/mus_musculus/Mus_musculus.GRCm39.110.gtf.gz"

# Download and extract genome files
genome_file = os.path.join(genome_dir, "Mus_musculus.GRCm39.dna.primary_assembly.fa")
gtf_file = os.path.join(genome_dir, "Mus_musculus.GRCm39.110.gtf")

# Download genome if not exists
if not os.path.exists(genome_file):
    print("Downloading genome file...")
    subprocess.run(['curl', '-o', f"{genome_file}.gz", genome_url], check=True)
    subprocess.run(['gunzip', '-f', f"{genome_file}.gz"], check=True)

# Download GTF if not exists
if not os.path.exists(gtf_file):
    print("Downloading GTF file...")
    subprocess.run(['curl', '-o', f"{gtf_file}.gz", gtf_url], check=True)
    subprocess.run(['gunzip', '-f', f"{gtf_file}.gz"], check=True)


In [23]:
# Run FastQC on trimmed files
print("Running FastQC on trimmed files...")
for r1_file in r1_files:
    sample_name = os.path.basename(r1_file).replace('_R1_001.fastq.gz', '')
    trimmed_r1 = os.path.join(output_dirs['trimmed'], f'{sample_name}_R1_trimmed.fastq.gz')
    trimmed_r2 = os.path.join(output_dirs['trimmed'], f'{sample_name}_R2_trimmed.fastq.gz')
    
    if os.path.exists(trimmed_r1) and os.path.exists(trimmed_r2):
        print(f"\nProcessing sample {sample_name}...")
        # Run FastQC
        cmd = ['fastqc', '-o', output_dirs['trimmed'], trimmed_r1, trimmed_r2]
        subprocess.run(cmd, check=True)
        print(f"✓ FastQC report generated for {sample_name}")
    else:
        print(f"✗ Files not found for sample {sample_name}")

print("\nFastQC reports are available in the 02_trimmed directory")

Running FastQC on trimmed files...

Processing sample 1...


python(62154) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


application/gzip
application/gzip


Started analysis of 1_R1_trimmed.fastq.gz
Approx 5% complete for 1_R1_trimmed.fastq.gz
Approx 10% complete for 1_R1_trimmed.fastq.gz
Approx 15% complete for 1_R1_trimmed.fastq.gz
Approx 20% complete for 1_R1_trimmed.fastq.gz
Approx 25% complete for 1_R1_trimmed.fastq.gz
Approx 30% complete for 1_R1_trimmed.fastq.gz
Approx 35% complete for 1_R1_trimmed.fastq.gz
Approx 40% complete for 1_R1_trimmed.fastq.gz
Approx 45% complete for 1_R1_trimmed.fastq.gz
Approx 50% complete for 1_R1_trimmed.fastq.gz
Approx 55% complete for 1_R1_trimmed.fastq.gz
Approx 60% complete for 1_R1_trimmed.fastq.gz
Approx 65% complete for 1_R1_trimmed.fastq.gz
Approx 70% complete for 1_R1_trimmed.fastq.gz
Approx 75% complete for 1_R1_trimmed.fastq.gz
Approx 80% complete for 1_R1_trimmed.fastq.gz
Approx 85% complete for 1_R1_trimmed.fastq.gz
Approx 90% complete for 1_R1_trimmed.fastq.gz
Approx 95% complete for 1_R1_trimmed.fastq.gz
Approx 100% complete for 1_R1_trimmed.fastq.gz


Analysis complete for 1_R1_trimmed.fastq.gz


...

application/gzip
application/gzip


Started analysis of 5_R1_trimmed.fastq.gz
Approx 5% complete for 5_R1_trimmed.fastq.gz
Approx 10% complete for 5_R1_trimmed.fastq.gz
Approx 15% complete for 5_R1_trimmed.fastq.gz
Approx 20% complete for 5_R1_trimmed.fastq.gz
Approx 25% complete for 5_R1_trimmed.fastq.gz
Approx 30% complete for 5_R1_trimmed.fastq.gz
Approx 35% complete for 5_R1_trimmed.fastq.gz
Approx 40% complete for 5_R1_trimmed.fastq.gz
Approx 45% complete for 5_R1_trimmed.fastq.gz
Approx 50% complete for 5_R1_trimmed.fastq.gz
Approx 55% complete for 5_R1_trimmed.fastq.gz
Approx 60% complete for 5_R1_trimmed.fastq.gz
Approx 65% complete for 5_R1_trimmed.fastq.gz
Approx 70% complete for 5_R1_trimmed.fastq.gz
Approx 75% complete for 5_R1_trimmed.fastq.gz
Approx 80% complete for 5_R1_trimmed.fastq.gz
Approx 85% complete for 5_R1_trimmed.fastq.gz
Approx 90% complete for 5_R1_trimmed.fastq.gz
Approx 95% complete for 5_R1_trimmed.fastq.gz


Analysis complete for 5_R1_trimmed.fastq.gz


Started analysis of 5_R2_trimmed.fastq.gz
Approx 5% complete for 5_R2_trimmed.fastq.gz
Approx 10% complete for 5_R2_trimmed.fastq.gz
Approx 15% complete for 5_R2_trimmed.fastq.gz
Approx 20% complete for 5_R2_trimmed.fastq.gz
Approx 25% complete for 5_R2_trimmed.fastq.gz
Approx 30% complete for 5_R2_trimmed.fastq.gz
Approx 35% complete for 5_R2_trimmed.fastq.gz
Approx 40% complete for 5_R2_trimmed.fastq.gz
Approx 45% complete for 5_R2_trimmed.fastq.gz
Approx 50% complete for 5_R2_trimmed.fastq.gz
Approx 55% complete for 5_R2_trimmed.fastq.gz
Approx 60% complete for 5_R2_trimmed.fastq.gz
Approx 65% complete for 5_R2_trimmed.fastq.gz
Approx 70% complete for 5_R2_trimmed.fastq.gz
Approx 75% complete for 5_R2_trimmed.fastq.gz
Approx 80% complete for 5_R2_trimmed.fastq.gz
Approx 85% complete for 5_R2_trimmed.fastq.gz
Approx 90% complete for 5_R2_trimmed.fastq.gz
Approx 95% complete for 5_R2_trimmed.fastq.gz


Analysis complete for 5_R2_trimmed.fastq.gz
✓ FastQC report generated for 5

FastQC reports are available in the 01_fastqc directory


In [6]:
# Create STAR index
index_dir = os.path.join(genome_dir, 'star_index')
os.makedirs(index_dir, exist_ok=True)


In [14]:

print("Creating STAR index...")
star_index_cmd = [
    'STAR',
    '--runMode', 'genomeGenerate',
    '--genomeDir', index_dir,
    '--genomeFastaFiles', os.path.join(genome_dir, 'genome.fa'),
    '--sjdbGTFfile', os.path.join(genome_dir, 'genes.gtf'),
    '--runThreadN', '4'
]

result = subprocess.run(star_index_cmd, capture_output=True, text=True)
if result.returncode == 0:
    print("STAR index created successfully")
else:
    print("Error creating STAR index:")
    print(result.stderr)

Creating STAR index...
STAR index created successfully


In [15]:
# Map trimmed reads to the genome
for r1_file in r1_files:
    sample_name = os.path.basename(r1_file).replace('_R1_001.fastq.gz', '')
    
    # Define input and output files
    trimmed_r1 = os.path.join(output_dirs['trimmed'], f'{sample_name}_R1_trimmed.fastq.gz')
    trimmed_r2 = os.path.join(output_dirs['trimmed'], f'{sample_name}_R2_trimmed.fastq.gz')
    out_dir = os.path.join(output_dirs['star'], sample_name)
    os.makedirs(out_dir, exist_ok=True)
    
    print(f"Mapping sample {sample_name}...")
    
    # STAR mapping command
    star_cmd = [
        'STAR',
        '--genomeDir', index_dir,
        '--readFilesIn', trimmed_r1, trimmed_r2,
        '--readFilesCommand', 'zcat',  # For reading gzipped files
        '--outFileNamePrefix', os.path.join(out_dir, f'{sample_name}_'),
        '--outSAMtype', 'BAM', 'SortedByCoordinate',
        '--runThreadN', '4',
        '--outBAMsortingThreadN', '4'
    ]
    
    # Run STAR
    result = subprocess.run(star_cmd, capture_output=True, text=True)
    
    if result.returncode == 0:
        print(f"Successfully mapped {sample_name}")
    else:
        print(f"Error mapping {sample_name}:")
        print(result.stderr)

Mapping sample 1...


python(51956) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Successfully mapped 1
Mapping sample 2...


python(52037) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Successfully mapped 2
Mapping sample 3...


...

Successfully mapped 3
Mapping sample 4...


python(52377) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Successfully mapped 4
Mapping sample 5...


python(52509) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Successfully mapped 5


In [ ]:
# Process each sample with STAR
import shutil  # Add this import at the top of your notebook

for r1_file in r1_files:
    sample_name = os.path.basename(r1_file).replace('_R1_001.fastq.gz', '')
    r2_file = r1_file.replace('_R1_001.fastq.gz', '_R2_001.fastq.gz')
    
    # Define input and output files
    trimmed_r1 = os.path.join(output_dirs['trimmed'], f'{sample_name}_R1_trimmed.fastq.gz')
    trimmed_r2 = os.path.join(output_dirs['trimmed'], f'{sample_name}_R2_trimmed.fastq.gz')
    out_dir = os.path.join(output_dirs['star'], sample_name)
    os.makedirs(out_dir, exist_ok=True)
    
    # Create temporary directory in Linux filesystem
    tmp_dir = os.path.expanduser(f'~/tmp_star/{sample_name}_tmp')
    
    print(f"\nProcessing sample {sample_name}...")
    
    # STAR alignment command
    star_cmd = [
        'STAR',
        '--genomeDir', index_dir,
        '--readFilesIn', trimmed_r1, trimmed_r2,
        '--readFilesCommand', 'zcat',
        '--outFileNamePrefix', os.path.join(out_dir, f'{sample_name}_'),
        '--outTmpDir', tmp_dir,
        '--outSAMtype', 'BAM', 'SortedByCoordinate',
        '--runThreadN', '2',  # Reduced threads
        '--genomeSAindexNbases', '12',  # Reduced from default 14
        '--outFilterMultimapNmax', '10',  # Reduced from 20
        '--outFilterMismatchNmax', '999',
        '--outFilterMismatchNoverReadLmax', '0.04',
        '--alignIntronMin', '20',
        '--alignIntronMax', '1000000',
        '--alignMatesGapMax', '1000000',
        '--alignSJoverhangMin', '8',
        '--alignSJDBoverhangMin', '1',
        '--outFilterType', 'BySJout',
        '--genomeLoad', 'LoadAndKeep',  # Keep genome in memory between runs
        '--limitBAMsortRAM', '2000000000',  # 2GB for BAM sorting
        '--limitGenomeGenerateRAM', '7000000000',  # Limit to 7GB RAM
    ]
    
    # Run STAR
    result = subprocess.run(star_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    
    if result.returncode == 0:
        print(f"✓ Successfully processed {sample_name}")
    else:
        print(f"✗ Error processing {sample_name}")
        print(result.stderr)

# Clean up temp directory after successful run
subprocess.run(["rm", "-rf", os.path.expanduser("~/tmp_star/*")])

print("\nSTAR alignment and counting complete!")


Processing sample 1...
